<a href="https://colab.research.google.com/github/cliche-niche/model-zoo-submissions/blob/main/ResNet/resNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers

In [2]:
(trainIm, trainLab), (testIm, testLab) = cifar10.load_data()
trainIm = trainIm / 255.0
testIm = testIm / 255.0
trainLab = tf.keras.utils.to_categorical(trainLab, 10)
testLab = tf.keras.utils.to_categorical(testLab, 10)

170508288/170498071 [==============================] - 2s 0us/step


In [3]:
#For callbacks
class mcb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
cb = mcb()

https://www.tensorflow.org/guide/keras/custom_layers_and_models

Two classes, `resId` and `resSh`, with reference from the link above were written.

`resId` is used to make a resNet block in which Identity shortcuts are used, padding is used to keep dimensions of residual the same as input (inp). They are then added together and a LeakyReLU layer is applied on them

`resSh` is used to make a resNet block in which Shortcuts are used, the size of input is made the same as residual by using a 1x1 convolution with `strides= (2, 2)`. The residual is obtained by first using a `3x3 convolution` (with padding) and `strides= (2, 2)`, followed by another `3x3 convolution` (with padding) but `strides=1` this time.

A layer for `BatchNormalization`, followed by a `ReLU` is used after every convolution, unless they're being added, in which case `ReLU` is applied after their addition

In [4]:
class resId(layers.Layer):
    def __init__(self, filters=64, kerSize=3):
        super(resId, self).__init__()
        self.co = layers.Conv2D(filters, kernel_size=kerSize, padding='same',
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.bn = layers.BatchNormalization()
        self.re = layers.Activation(activations.relu)
    def call(self, inp):
        x = inp
        id = inp
        x = self.co(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co(x)
        x = self.bn(x)
        x = layers.add([x, id])
        x = self.re(x)
        return x
 
class resSh(layers.Layer):
    def __init__(self, filters, kerSize=3, stride=(2, 2)):
        super(resSh, self).__init__()
        self.co1 = layers.Conv2D(filters, kernel_size=kerSize, padding='same', strides=stride,
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.bn = layers.BatchNormalization()
        self.re = layers.Activation(activations.relu)
        self.co2 = layers.Conv2D(filters, kernel_size=kerSize, padding='same',
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.co = layers.Conv2D(filters, (1, 1), strides=stride, padding='same')
    def call(self, inp):
        x = inp
        sh = inp
        x = self.co1(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co2(x)
        x = self.bn(x)
        sh = self.co(sh)
        x = layers.add([x, sh])
        x = self.re(x)
        return x

This is a resNet model based on the paper:
https://arxiv.org/abs/1512.03385

It has some additional `Dropout` layers to obtain a better accuracy on test data, rather than without the layers.

In [5]:
class resNet(tf.keras.Model):
    def __init__(self):
        super(resNet, self).__init__()
        self.co1 = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')
        self.mp1 = layers.MaxPooling2D((2, 2), padding='same')

        self.ri1 = resId(filters=64, kerSize=3)
        self.rs1 = resSh(filters=128, kerSize=3, stride=(2, 2))
        self.ri2 = resId(filters=128, kerSize=3)
        self.rs2 = resSh(filters=256, kerSize=3, stride=(2, 2))
        self.ri3 = resId(filters=256, kerSize=3)
        self.rs3 = resSh(filters=512, kerSize=3, stride=(2, 2))
        self.ri4 = resId(filters=512, kerSize=3)

        self.gap = layers.GlobalAveragePooling2D()
        self.fla = layers.Flatten()
        self.end = layers.Dense(10, activation="softmax", kernel_initializer='he_normal')

    def call(self, inp):
        x = inp
        x = self.co1(x)
        x = self.mp1(x)
        for i in range(3):
            x = self.ri1(x)
        x = self.rs1(x)
  #      x = layers.Dropout(0.2)(x)
        for i in range(3):
            x = self.ri2(x)
        x = self.rs2(x)
  #      x = layers.Dropout(0.2)(x)
        for i in range(5):
            x = self.ri3(x)
        x = self.rs3(x)
 #       x = layers.Dropout(0.2)(x)
        x = self.ri4(x)
        x = self.ri4(x)
   #     x = layers.Dropout(0.2)(x)

        x = self.gap(x)
        x = self.fla(x)
        x = self.end(x)
        return(x)

    def model(self):
        x = layers.Input(shape=(32, 32, 3))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        zca_epsilon=1e-06,
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',
        horizontal_flip=True)
datagen.fit(trainIm)

In [7]:
model = resNet()
print(model.model().summary())
model.compile(  optimizer = tf.keras.optimizers.Adam(lr=1e-3) ,
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 8, 8, 64)     0           conv2d[0][0]                     
__________________________________________________________________________________________________
res_id (resId)                  (None, 8, 8, 64)     37184       max_pooling2d[0][0]              
                                                                 res_id[0][0]                 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.fit_generator(datagen.flow(trainIm, trainLab, batch_size=32), epochs=100, callbacks=[cb], workers=4, validation_data= (testIm, testLab))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
1563/1563 [==============================] - 64s 38ms/step - loss: 2.1346 - accuracy: 0.3528 - val_loss: 2.5000 - val_accuracy: 0.1726
Epoch 2/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.7989 - accuracy: 0.4452 - val_loss: 2.2978 - val_accuracy: 0.2475
Epoch 3/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.6187 - accuracy: 0.4953 - val_loss: 2.2378 - val_accuracy: 0.2636
Epoch 4/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.5300 - accuracy: 0.5276 - val_loss: 2.1461 - val_accuracy: 0.2489
Epoch 5/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.4052 - accuracy: 0.5675 - val_loss: 1.8645 - val_accuracy: 0.3944
Epoch 6/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.3897 - accuracy: 0.5810 - val_loss: 1.7809 - val_accuracy: 0.4162
Epoch 7/100
1563/1563 [==============================] - 59s 38ms/step - loss: 1.2891 - accuracy: 0.6158 - val_loss: 1

In [ ]:
model.evaluate(testIm, testLab)

In [ ]:
train_loss, train_acc = model.evaluate(trainIm, trainLab)
test_loss, test_acc = model.evaluate(testIm, testLab)
print("Training loss:", train_loss)
print("Training accuracy:", train_acc)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)